In [3]:
import pandas as pd

In [15]:
# read in trajectories data from parquet from data/_11_analysis_&_conclusions/trajectories.parquet
trajectories = pd.read_parquet('../data/_11_analysis_&_conclusions/trajectories.parquet')

display(trajectories)


,segment_id,cluster,is_most_typical,is_most_extreme,icao,start_timestamp,point_idx,x,y,z,heading_deg,ground_speed,time_sec
0,998460,1,False,False,46a5e2,2024-08-01 20:43:54.340000+00:00,0,0.00,0.00,38975.00,113.80,490.70,0.00
1,998460,1,False,False,46a5e2,2024-08-01 20:43:54.340000+00:00,1,8849.21,-3902.97,38975.00,113.80,490.70,19.71
2,998460,1,False,False,46a5e2,2024-08-01 20:43:54.340000+00:00,2,26601.51,-11732.67,38975.00,113.80,490.70,39.54
3,998460,1,False,False,46a5e2,2024-08-01 20:43:54.340000+00:00,3,53374.38,-23596.77,38975.00,113.90,492.00,59.52
4,998460,1,False,False,46a5e2,2024-08-01 20:43:54.340000+00:00,4,88828.64,-39307.93,38975.00,113.90,492.00,78.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
451995,963898,-1,False,False,458a59,2024-08-01 11:15:44.270000+00:00,45,-278808.40,507029.78,700.00,275.70,100.50,202.42
451996,963898,-1,False,False,458a59,2024-08-01 11:15:44.270000+00:00,46,-298821.92,505208.40,700.00,264.80,98.40,204.23
451997,963898,-1,False,False,458a59,2024-08-01 11:15:44.270000+00:00,47,-318644.61,501283.40,600.00,258.80,97.90,206.41
451998,963898,-1,False,False,458a59,2024-08-01 11:15:44.270000+00:00,48,-339034.00,497023.84,500.00,258.20,98.10,212.33


In [18]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Get the first segment_id
first_segment = trajectories['segment_id'].unique()[0]

# Filter data for first segment
segment_data = trajectories[trajectories['segment_id'] == first_segment]

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=segment_data['x'],
    y=segment_data['y'],
    z=segment_data['z'],
    mode='lines+markers',
    line=dict(
        width=2,
        color='blue'
    ),
    marker=dict(
        size=2,
        color='blue',
        opacity=0.8  # Added some transparency
    )
)])

# Update layout for better visualization
fig.update_layout(
    title=f'Flight Path - Segment {first_segment}',
    scene=dict(
        camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=1.5, y=1.5, z=1.5)  # Adjusted camera position
        ),
        aspectmode='cube',  # Forces a cubic aspect ratio
        xaxis=dict(
            title='X',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            title='Y',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            title='Z',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    ),
    width=1000,
    height=800
)

# Display interactive plot
fig.show()

# Save static version for GitHub
pio.write_image(fig, "flight_path.png")

# Optional: Print the range of values for each axis to check scale
print(f"X range: {segment_data['x'].min():.2f} to {segment_data['x'].max():.2f}")
print(f"Y range: {segment_data['y'].min():.2f} to {segment_data['y'].max():.2f}")
print(f"Z range: {segment_data['z'].min():.2f} to {segment_data['z'].max():.2f}")

X range: 0.00 to 10885158.10
Y range: -3575510.48 to 0.00
Z range: 38975.00 to 39050.00


In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import random
import plotly.express as px  # for the color sequence

# Filter for cluster 1
cluster_data = trajectories[trajectories['cluster'] == 1]

# Convert decimal columns to float if needed
numeric_columns = ['x', 'y', 'z']
for col in numeric_columns:
    if cluster_data[col].dtype == 'object':
        cluster_data[col] = cluster_data[col].astype(float)

# Get unique segment IDs for cluster 1
unique_segments = cluster_data['segment_id'].unique()

# Get random sample of up to 50 segment IDs
n_segments = min(50, len(unique_segments))
selected_segments = np.random.choice(unique_segments, size=n_segments, replace=False)

# Calculate overall ranges for consistent scaling
x_min, x_max = cluster_data['x'].min(), cluster_data['x'].max()
y_min, y_max = cluster_data['y'].min(), cluster_data['y'].max()
z_min, z_max = cluster_data['z'].min(), cluster_data['z'].max()
max_range = max(x_max - x_min, y_max - y_min, z_max - z_min)

# Create a color sequence
colors = px.colors.qualitative.Set3  # Using Set3 palette which has good visibility for multiple traces
# If we need more colors than available in the palette, we'll cycle through them
colors = colors * (n_segments // len(colors) + 1)

# Create the 3D scatter plot
fig = go.Figure()

# Add each trajectory as a separate trace
for idx, segment_id in enumerate(selected_segments):
    segment_data = cluster_data[cluster_data['segment_id'] == segment_id]
    
    fig.add_trace(go.Scatter3d(
        x=segment_data['x'],
        y=segment_data['y'],
        z=segment_data['z'],
        mode='lines',
        line=dict(
            width=2,
            color=colors[idx]
        ),
        opacity=0.8,  # Increased opacity a bit since we have different colors
        name=f'Segment {segment_id}'
    ))

# Update layout for better visualization
fig.update_layout(
    title=f'Flight Paths - Cluster 1 (Random {n_segments} Segments)',
    scene=dict(
        camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=1.5, y=1.5, z=1.5)
        ),
        aspectmode='cube',
        xaxis=dict(
            title='X',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)',
            range=[float(x_min - max_range*0.1), float(x_max + max_range*0.1)]
        ),
        yaxis=dict(
            title='Y',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)',
            range=[float(y_min - max_range*0.1), float(y_max + max_range*0.1)]
        ),
        zaxis=dict(
            title='Z',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)',
            range=[float(z_min - max_range*0.1), float(z_max + max_range*0.1)]
        )
    ),
    width=1000,
    height=800,
    showlegend=True
)

# Display interactive plot
fig.show()

# Save static version
pio.write_image(fig, "cluster1_paths_colored.png")

print(f"Total number of segments in cluster 1: {len(unique_segments)}")
print(f"Number of segments plotted: {n_segments}")

NameError: name 'trajectories' is not defined